# Modeling

In [4]:
import cv2
import os
from collections import defaultdict
import threading
import time
import shutil

import tensorflow as tf
import keras
import tensorflow_model_optimization as tfmot
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Flatten, Dense
from tensorflow.keras.layers import Activation, MaxPooling2D, Concatenate, UpSampling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.applications.resnet import ResNet50
from keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

MAX_THREADS = 6
semaphore = threading.Semaphore(MAX_THREADS)
threads = []

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus :
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
# Function to get polygon annotations for each building
def get_polygon_annotations(feature) :
    poly_annotations = {}
    
    for feat in feature :
        # Convert string format to polygon object
        feat_shape = wkt.loads(feat['wkt'])
        
        # Extract coordinates of the polygon 
        coords = list(mapping(feat_shape)['coordinates'][0])
        
        # Store unique id and coordinates for each building as a Numpy array
        poly_annotations[feat['properties']['uid']] = (np.array(coords, np.int32))
        
    return poly_annotations

# Function to get image dimensions 
def get_image_dimensions(image_folder, filename) :

    image_path = os.path.join(image_folder, filename)
    
    # Read and convert the image to a numpy array to get the size
    image = io.imread(image_path)
    image_arr = np.array(image)
    image_size = image_arr.shape
    
    return image_size

# Function to locate buildings using polygon annotations
def mask_polygons(size, poly_annotations) :

    # Creating black empty mask image 
    mask_img = np.zeros(size, dtype=np.uint8)
    
    for points in poly_annotations :
        
        # Creating empty mask image to hold one polygon 
        blank_img = np.zeros(size, dtype=np.uint8)
        
        # Extract list of points to locate the building
        poly = poly_annotations[points]
        
        # Fill the blank image with polygon points 
        cv2.fillPoly(blank_img, [poly], (1,1,1))
        
        # Draw the border around the polygon
        cv2.polylines(blank_img, [poly], isClosed=True, color=(2, 2, 2), thickness=2)
        
        # Adding the filled image to the main mask image
        mask_img += blank_img
        
    # Set pixel values greater than 2 to 0 to retain non-overlapping areas
    mask_img[mask_img > 2] = 0
    
    # Convert non-overlapping areas to white to locate buildings
    mask_img[mask_img == 1] = 255
    mask_img[mask_img == 2] = 127
    
    return mask_img

# Function to resize images to a standard scale
def resize_images(image_paths, masked=False, target_size=(256, 256)):
    resized_images=[]
    
    for path in image_paths:
        image = cv2.imread(path)
        # Resize the image to the target dimensions
        image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
        
        if(not masked):
            # Normalize pixel values to the range [0, 1]
            image = image.astype(np.float32) / 255.0
        else:
            image = (image > 0).astype(np.uint8)
            if image.shape[2] == 3:
                # Convert from 3 channels to 1 channel 
                image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
                # Expand dimensions to single channel shape
                image = np.expand_dims(image, axis=-1)
        resized_images.append(image)
    
    return resized_images

def create_masked_image(feature, image_name, images_dir): 
    with semaphore:
        poly_annotations = get_polygon_annotations(feature)
        image_size = get_image_dimensions(images_dir, image_name)
        mask_image = mask_polygons(image_size, poly_annotations)
            
        # Save the mask image to the output folder
        filename = image_name.split('.')[0]
        masked_dirpath = os.path.join(masked_dir, f"{filename}_mask.png")
        cv2.imwrite(masked_dirpath, mask_image)

In [3]:
# Function to extract damage type for each building
def extract_damage_type(feature) :
    polygon_data = []
    
    damage_label_encoding = defaultdict(int)
    damage_label_encoding['no-damage'] = 0
    damage_label_encoding['minor-damage'] = 1
    damage_label_encoding['major-damage'] = 2
    damage_label_encoding['destroyed'] = 3
    
    for feat in feature:
        # Extract uid and damage type
        uid = feat['properties'].get('uid', None) 
        damage_type = feat['properties'].get('subtype', 'no-damage')
        
        if uid:
            uid += ".png"
            # Extract polygon coordinates
            poly_geom = wkt.loads(feat['wkt'])
            polygon_points = np.array(list(poly_geom.exterior.coords))
            
            if polygon_points.size > 0:
                polygon_data.append({"uid": uid,
                                     "damage_type": damage_label_encoding[damage_type],
                                     "polygon_points": polygon_points})
        
    return polygon_data
    
def process_post_img(image_dir, image_filename, polygon_pts, scale_pct) :
    
    image_path = os.path.join(image_dir, image_filename)
    # Image dimensions
    image = io.imread(image_path)
    img_array = np.array(image)
    height, width, _ = img_array.shape

    # Compute bounding box using X and Y coordinates
    xcoords = polygon_pts[:, 0]
    ycoords = polygon_pts[:, 1]
    xmin, xmax = np.min(xcoords), np.max(xcoords)
    ymin, ymax = np.min(ycoords), np.max(ycoords)

    # Width and height 
    xdiff = xmax - xmin
    ydiff = ymax - ymin

    #Extend image by scale percentage
    xmin = max(int(xmin - (xdiff * scale_pct)), 0)
    xmax = min(int(xmax + (xdiff * scale_pct)), width)
    ymin = max(int(ymin - (ydiff * scale_pct)), 0)
    ymax = min(int(ymax + (ydiff * scale_pct)), height)

    return img_array[ymin:ymax, xmin:xmax, :]

def create_cropped_image(poly_data, image_name, images_dir): 
    with semaphore:
        for data in poly_data:
            uid = data['uid']
            polygon_pts = data['polygon_points']
            processed_img = process_post_img(images_dir, image_name, polygon_pts, 0.8)
            output_path = os.path.join(cropped_dir, f"{uid}")
            cv2.imwrite(output_path, processed_img)

In [4]:
# Directories for masked and cropped images
masked_dir = os.path.join(os.pardir, "masks")
cropped_dir = os.path.join(os.pardir, "cropped_images")

def preprocess_data(hurricane_pre_df, hurricane_post_df, images_dir):
    total_buildings = 0
    pre_hurricane_mask_images = []
    polygon_data = []
    threads = []
    
    def process_pre_and_post(pre_feature, pre_image_name, post_feature, post_image_name):
        """Handles processing of both pre- and post-hurricane images in a single thread."""
        create_masked_image(pre_feature, pre_image_name, images_dir)

        if post_feature:
            nonlocal total_buildings
            poly_data = extract_damage_type(post_feature)
            create_cropped_image(poly_data, post_image_name, images_dir)
            polygon_data.extend(poly_data)
            total_buildings += len(post_feature)
            
    if not os.path.exists(masked_dir) :
        os.makedirs(masked_dir)

    masked_files_count = len([f for f in os.listdir(masked_dir) if os.path.isfile(os.path.join(masked_dir, f))])
    if masked_files_count != len(hurricane_pre_df):

        for pre_row, post_row in zip(hurricane_pre_df.itertuples(index=False, name="Pandas"), hurricane_post_df.itertuples(index=False, name="Pandas")):
            pre_feature = pre_row.xy
            post_feature = post_row.xy
            pre_image_name = pre_row.img_name
            post_image_name = post_row.img_name
    
            # Start thread for pre- and post-image processing
            t = threading.Thread(target=process_pre_and_post, args=(pre_feature, pre_image_name, post_feature, post_image_name))
            threads.append(t)
            t.start()

        # Ensure all threads are completed
        for t in threads:
            t.join()

        print(f"Total pre-disaster mask images: {len(pre_hurricane_mask_images)}")
        print(f"\nTotal buildings processed: {total_buildings}")
        
    else:
        for index, post_row in hurricane_post_df.iterrows() :
            post_feature = post_row['xy']
            poly_data = extract_damage_type(post_feature)
            polygon_data.extend(poly_data)

    pre_hurricane_mask_images = [image for image in glob.iglob(f'{masked_dir}/*') if image.endswith(".png")]

    pre_resized_images = resize_images(pre_hurricane_mask_images, masked=False)
    pre_mask_resized_images = resize_images(pre_hurricane_mask_images, masked=True)

    return pre_resized_images, pre_mask_resized_images, polygon_data


In [5]:
def split_data(pre_resized_img, pre_mask_resized_img):
    # Convert lists to Numpy arrays 
    X = np.array(pre_resized_img)
    y = np.array(pre_mask_resized_img)
    print(X.shape)
    print(y.shape)
    # Split into training and test sets
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_valid, y_train, y_valid

def classification_split_data(df):
    train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)
    #valid_df, test_df = train_test_split(valid_df, test_size=0.1, random_state=42)    
    print(train_df.shape)
    print(valid_df.shape)
    #print(test_df.shape)
    return train_df, valid_df

In [6]:
def build_fcn_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    
    # Decoder
    x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)  # 1-channel output for binary mask

    model = tf.keras.Model(inputs, outputs)
    return model

In [7]:
def Unet_conv_block(inputs, num_filters) :
    """Convolution layer with 3x3 filter 
    followed by BatchNormalization 
    and ReLU activation"""
    
    x = Conv2D(num_filters, (3, 3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def Unet_encoder_block(inputs, num_filters) :
    
    x = Unet_conv_block(inputs, num_filters)
    # Max pooling with 2x2 filter
    x = MaxPooling2D((2,2))(x)
    
    return x

def Unet_decoder_block(inputs, num_filters, skip) :
    x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding="same")(inputs)
    
    # Check the dimension of upsampled output and skip connection
    if x.shape[1] != skip.shape[1] or x.shape[2] != skip.shape[2]:
        skip = UpSampling2D((2, 2))(skip)
        
    x = Concatenate()([x, skip])
    x = Unet_conv_block(x, num_filters)
    
    return x

def build_unet_model(input_shape) :
    inputs = tf.keras.Input(input_shape)
    
    # Encoders
    encoder1 = Unet_encoder_block(inputs, 64)
    encoder2 = Unet_encoder_block(encoder1, 128)
    encoder3 = Unet_encoder_block(encoder2, 256)
    encoder4 = Unet_encoder_block(encoder3, 512)
    
    # Bottleneck
    bridge1 = Unet_conv_block(encoder4, 1024)
    
    # Decoders
    decoder1 = Unet_decoder_block(bridge1, 512, encoder4)
    decoder2 = Unet_decoder_block(decoder1, 256, encoder3)
    decoder3 = Unet_decoder_block(decoder2, 128, encoder2)
    decoder4 = Unet_decoder_block(decoder3, 64, encoder1)
    
    # Output
    outputs = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(decoder4)
    
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")
    
    return unet_model

In [8]:
# def data_gen():
#     # Data generators
#     train_datagen = ImageDataGenerator(rescale=1/255.)
#     val_datagen = ImageDataGenerator(rescale=1/255.)
#     # Flow data from dataframe
#     train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
#                                                         directory=cropped_dir,
#                                                         x_col='building_uid',
#                                                         y_col='labels',
#                                                         target_size=(128, 128),
#                                                         batch_size=32,
#                                                         seed=123,
#                                                         class_mode="categorical")
    
#     val_generator = val_datagen.flow_from_dataframe(dataframe=valid_df,
#                                                     directory=cropped_dir,
#                                                     x_col='building_uid',
#                                                     y_col='labels',
#                                                     target_size=(128, 128),
#                                                     batch_size=32,
#                                                     shuffle=False,
#                                                     seed=123,
#                                                     class_mode="categorical")

#     return train_generator, val_generator

def data_gen(target_size=(128,128),  batch_size=16):
    # Data generators
    train_datagen = ImageDataGenerator(
        rescale=1/255.,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    val_datagen = ImageDataGenerator(rescale=1/255.)

    # Flow data from dataframe
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=cropped_dir,
        x_col='building_uid',
        y_col='labels',
        target_size=target_size,
        batch_size=batch_size,
        seed=123,
        class_mode="categorical"
    )
    
    val_generator = val_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=cropped_dir,
        x_col='building_uid',
        y_col='labels',
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False,
        seed=123,
        class_mode="categorical"
    )

    return train_generator, val_generator

In [9]:
def build_resnet50_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    pretrained_resnet50 = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    
    for layer in pretrained_resnet50.layers:
        layer.trainable = False
        
    # Custom CNN layers
    # x = Conv2D(32, (5, 5), activation='relu', padding='same')(inputs)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # x = Flatten()(x)
    x = Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(224, 224, 3))(inputs)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

    x = Flatten()(x)
    resnet50_model = pretrained_resnet50(inputs)
    resnet50_model = Flatten()(resnet50_model)
    
    concated_layers = Concatenate()([x, resnet50_model])
    
    concated_layers = Dense(2024, activation='relu')(concated_layers)
    concated_layers = Dense(524, activation='relu')(concated_layers)
    concated_layers = Dense(124, activation='relu')(concated_layers)
    outputs = Dense(4, activation='relu')(concated_layers)

    model = tf.keras.Model(inputs, outputs)
    # prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    # model = prune_low_magnitude(model)
    
    return model

# def build_resnet50_model(input_shape):
#     inputs = tf.keras.Input(shape=input_shape)
    
#     # Load pretrained ResNet50
#     pretrained_resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    
#     # Freeze earlier layers
#     for layer in pretrained_resnet.layers[:-20]:
#         layer.trainable = False
    
#     # Add ResNet branch
#     resnet_output = pretrained_resnet(inputs)
#     resnet_output = Flatten()(resnet_output)
    
#     # Custom convolutional layers
#     x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
#     x = BatchNormalization()(x)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
    
#     x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
    
#     x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
    
#     x = Flatten()(x)
    
#     # Concatenate features
#     concated_layers = Concatenate()([x, resnet_output])
    
#     # Dense layers with regularization and dropout
#     concated_layers = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(concated_layers)
#     concated_layers = Dropout(0.5)(concated_layers)
#     concated_layers = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(concated_layers)
#     concated_layers = Dropout(0.5)(concated_layers)
#     outputs = Dense(4, activation='softmax')(concated_layers)
    
#     model = tf.keras.Model(inputs, outputs)
    
#     return model

In [10]:
# def build_mobilenet_model(input_shape):
#     inputs = tf.keras.Input(shape=input_shape)
    
#     pretrained_mobilenet = MobileNet(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
    
#     for layer in pretrained_mobilenet.layers:
#         layer.trainable = False
        
#     x = Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
#     x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

#     x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
#     x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)
    
#     x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
#     x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)
    
#     x = Flatten()(x)
    
#     mobilenet_model = pretrained_mobilenet(inputs)
#     mobilenet_model = Flatten()(mobilenet_model)
    
#     concated_layers = Concatenate()([x, mobilenet_model])
    
#     concated_layers = Dense(2024, activation='relu')(concated_layers)
#     concated_layers = Dense(524, activation='relu')(concated_layers)
#     concated_layers = Dense(124, activation='relu')(concated_layers)
#     outputs = Dense(4, activation='softmax')(concated_layers)
    
#     model = tf.keras.Model(inputs, outputs)
    
#     return model

def build_mobilenet_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Load pretrained MobileNet
    pretrained_mobilenet = MobileNet(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    
    # Unfreeze last 20 layers for fine-tuning
    for layer in pretrained_mobilenet.layers[:-20]:
        layer.trainable = False
    
    # Custom convolutional layers
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Flatten()(x)
    
    # MobileNet branch
    mobilenet_output = pretrained_mobilenet(inputs)
    mobilenet_output = Flatten()(mobilenet_output)
    
    # Concatenate features
    concated_layers = Concatenate()([x, mobilenet_output])
    
    # Dense layers with regularization and dropout
    concated_layers = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(concated_layers)
    concated_layers = Dropout(0.5)(concated_layers)
    concated_layers = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(concated_layers)
    concated_layers = Dropout(0.5)(concated_layers)
    outputs = Dense(4, activation='softmax')(concated_layers)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [2]:
def build_CNN_model(input_shape, num_classes) :
    model = tf.keras.models.Sequential([
        # Convolutional Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        
        # Convolutional Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        
        # Convolutional Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        
        # Fully Connected Layer
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Dropout to reduce overfitting
        Dense(num_classes, activation='softmax')  # Output layer
    ])
    return model

In [3]:
def lr_schedule(epoch):
    return 1e-3 * 0.1**(epoch // 10)

# Callbacks
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
    
def train_FCN_model(batch_size=16, epochs=50):
    with tf.device('/cpu:0'):
        # Instantiate the model
        fcn_model = build_fcn_model((256, 256, 3))
        # Compile the model
        fcn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        # Training the model 
    with tf.device('/gpu:0'):
        # fcn_model.summmary()
        history = fcn_model.fit(X_train, y_train, 
                            validation_data=(X_valid, y_valid),
                            epochs=epochs,
                            batch_size=batch_size)
        
        fcn_model.save('model/FCN/FCN_model.keras')

def train_Unet_model(batch_size=16, epochs=50):
    with tf.device('/cpu:0'):
        # Instantiate the model
        unet_model = build_unet_model((256, 256, 3))
        # Compile the model
        unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    with tf.device('/gpu:0'):
        unet_model.summary()
        history = unet_model.fit(X_train, y_train, 
                        validation_data=(X_valid, y_valid),
                        epochs=epochs,
                        batch_size=batch_size)
        unet_model.save('model/Unet/Unet_model.keras')

def train_ResNet50_model(train_generator, val_generator, batch_size=64, epochs=50) :
    with tf.device('/cpu:0'):
        # Instantiate the model
        resnet50_model = build_resnet50_model((224, 224, 3))
        # Compile the model
        resnet50_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    with tf.device('/gpu:0'):
        history = resnet50_model.fit(train_generator, 
                            validation_data=val_generator,
                            epochs=epochs,
                            batch_size=batch_size,
                            callbacks=[early_stopping, lr_scheduler])
                            # steps_per_epoch=len(train_generator))
    
        resnet50_model.save('model/Resnet50/Resnet50_model.keras')

def train_MobileNet_model(train_generator, val_generator, batch_size=64, epochs=50) :
    samples = train_df['building_uid'].count()
    steps = np.ceil(samples/batch_size)
    with tf.device('/cpu:0'):
        # Instantiate the model
        mobilenet_model = build_mobilenet_model((224, 224, 3))
        # Compile the model
        mobilenet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    with tf.device('/gpu:0'):   
        # history = mobilenet_model.fit(train_generator, 
        #                     validation_data=val_generator,
        #                     epochs=epochs,
        #                     batch_size=batch_size,
        #                     callbacks=[early_stopping, lr_scheduler])
                            # steps_per_epoch=len(train_generator))
        history = mobilenet_model.fit_generator(generator=train_generator,
                            steps_per_epoch=steps,
                            epochs=epochs,
                            workers=4,
                            use_multiprocessing=False,
                            callbacks=[early_stopping, lr_scheduler],
                            verbose=1)

        mobilenet_model.save('model/MobileNet/MobileNet_model.keras')

def train_CNN_model(train_generator, val_generator, batch_size=16, epochs=50):   
    with tf.device('/cpu:0'):
        # Instantiate the model
        CNN_model = build_CNN_model((224, 224, 3), 4)
        # Compile the model
        CNN_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    with tf.device('/gpu:0'):
        history = CNN_model.fit(train_generator, 
                                validation_data=val_generator,
                                epochs=epochs,
                                batch_size=batch_size,
                                callbacks=[early_stopping, lr_scheduler])
        CNN_model.save('model/simpleCNN/simpleCNN_model.keras')